# CoinMetrics Case Study

Objective - to evaluate skills and abilities in multiple ways:
1. importing data
2. wrangling data
3. exploring data
4. analysis
5. modelling
6. communicating results

Provide:
1. A written explanation of how to approach the problem
2. Present the beginning phases of implementation using coin metrics data

Of the four options made available in the case study, option 3 was chosen

### Advocating for CoinMetric's data

Produce quality research that is of value to potential clients (doesn’t have to be complete) with a particular focus on network data

### Initial ideas

My first rough ideas were: 
1. comparing different Bitcoin based chains, (BTC, BCH, LTC, BSV) to test the influence of whales and compare this to their respective claims to be a SoV  or alternative to cash.
2. developing some of the research by Willy Woo. I find his research particularly interesting. Particularly, 
    1. days destroyed, 
    2. hodl waves, 
    3. thermo cap, 
    4. average cap. 
 
I think the following ideas are also interesting and worth investigating, but not possible within the scope of this exercise:
 
1. Tracking the number of twitter followers of various crypto-twitter thought leaders and celebrities to test the hypothesis that *"an increase in follower numbers shows that new retail investors are entering crypto-markets, and an increase in price is expected soon"*
 
 Thought leaders / crypto celebrities could be further grouped by what types of coins they speak about most  - SoV, smart contracts, DeFi, etc. 
 
 Weibo could be analysed as well as Twitter to understand Chinese markets, Korean twitter could be analysed for the Korean retail market, etc. 

2. I have an existing side project which has the goal of using a recurrent neural net to predict BTC price movements. The app (model, stored data, data pipeline, visualization of results) will run autonomously on Google Cloud Platform. Candle data is consumed from CoinAPI.io and stored in BigQuery. 

 Technical indicators will be calculated and used as additional factors to the model. Sentiment analysis from news outlets (Bloomberg, FT) would be added later. 

 The model would be written using TensorFlow, and the BigQuery tables names would use BQ's date format capabilites. This would make the project faster and cheaper. 

### 1. Testing the influence of whales on BTC forks and comparing to each chains claims e.g. as a store of value or alternative to cash

If a country has a much lower median income than mean income, it probably has high income inequality. 

Similarly, if a chain has a much smaller median transaction size than mean transaction size, it probably is not used by regular users and is controlled mostly be whales. 

This would contradict any claims the fork makes to being a form of digital cash. 

In [2]:
# TxTfrValMeanUSD - The sum USD value of native units transferred divided by the count of transfers (i.e., the mean "size" in USD of a transfer) that interval.

# TxTfrValMedUSD - The median USD value transferred per transfer (i.e., the median "size" in USD of a transfer) that interval.

# TxTfrValUSD - The sum USD value of all native units transferred (i.e., the aggregate size in USD of all transfers) that interval.

# Chains: BTC, BCH, BSV, LTC, DOGE

In [66]:
import requests
import json

import pandas as pd

In [61]:
def get_metricdata(asset_id, payload):
    url = f'https://community-api.coinmetrics.io/v2/assets/{asset_id}/metricdata'
    response = requests.get(
        url=url,
        params=payload
    )
    
    if response.status_code == 200:
        print(f'{asset_id} - success!')
        return json.loads(response.content.decode('utf-8'))
    else:
        print(f'status_code: {response.status_code}')
        return None

In [62]:
payload = {
    'metrics':  'PriceUSD,'+
                'TxTfrValMeanUSD,'+
                'TxTfrValMedUSD,'+
                'TxTfrValUSD',
    'start': '2018-09-01',
}

asset_list = ['btc', 'ltc', 'bch', 'bsv', 'doge']
data = {}
for asset in asset_list:
    data[asset] = get_metricdata(asset, payload)

btc - success!
ltc - success!
bch - success!
bsv - success!
doge - success!


In [124]:
data.keys()

dict_keys(['btc', 'ltc', 'bch', 'bsv', 'doge'])

In [125]:
dataframes = {}
cols = ['PriceUSD', 'TxTfrValMeanUSD', 'TxTfrValMedUSD', 'TxTfrValUSD']
for asset in data.keys():
    values = [ each['values'] for each in data[asset]['metricData']['series']]
    index = [ each['time'] for each in data[asset]['metricData']['series']]
    
    df = pd.DataFrame.from_records(values, columns = cols)
    df.index = pd.to_datetime(index, infer_datetime_format=True).date
    
    dataframes[asset] = df

In [132]:
dataframes['bsv'].sample(20)

PriceUSD                    TxTfrValMeanUSD  \
2019-05-28  122.08763368430292   5632.949328400788018078981840876   
2018-11-19  61.035951584272745  14.442474787643060165148643776715   
2019-02-18   67.63729227663508   5641.244216931395050927051832048   
2019-01-22   75.53427260030611     9088.9834128611718959265671118   
2019-07-09   204.7523694682079         1849.830732362874688970462   
2019-02-01   64.05142613902203    7080.53702680854983052498618087   
2018-12-23   106.9936690615481    47955.5586417154296650549130351   
2019-08-23    135.367137927592           210.54398729948295778208   
2019-03-13   64.39860281528094    4734.27351323925094022614620324   
2019-06-22  238.59589105252016        7506.6940965296368206146096   
2019-06-20   219.6797928775124         6528.465914314931741607188   
2019-04-08   83.17038237378495   815.7470574802568990149131066165   
2019-03-03   64.57798569307359   5547.273501500905665038720609544   
2019-03-15   67.19781383064858   11076.75439279511766256722839616   
2018-12-25   92.97623744043382   179.6086934335388488425608387488   
2019-06-01  189.12881659688367        6077.9394571420771024807119   
2019-02-23   67.54842854602741   6491.182288500069238995607925525   
2019-04-30   53.18651922791556    6092.97523973778359654371123632   
2019-02-11    64.1550669670929     8559.0433328617079554083885213   
2019-04-12   71.38631021234552   13899.07129860390453136867013832   

                       TxTfrValMedUSD                       TxTfrValUSD  
2019-05-28   0.0688769594193363353472  252553283.1388493673558174392844  
2018-11-19  0.00218813886429617790825     24730557.26014583939362501977  
2019-02-18  410.180369729422346623666   144968693.886703005656465352338  
2019-01-22      0.0007553427260030611   91671486.7021180305161003889059  
2019-07-09    0.032772664257081356474   191790450.361708721194393772229  
2019-02-01         0.6405142613902203   37965839.5377472970010977084292  
2018-12-23   3.3976048248149269173795    191438590.09772779921049749208  
2019-08-23          0.947569965493144     26559492.32536602900740051936  
2019-03-13       0.051518882252224752   41997740.3359453908404383571604  
2019-06-22         23.859589105252016  482627883.5231750907254709822448  
2019-06-20    7.894387773670104692474   396832800.547831347839961360696  
2019-04-08     0.00069863121193979358   130376773.461782097746419651416  
2019-03-03   2.3509195934656435461165   48311204.9245713800491006545009  
2019-03-15   3.8477071732327776081378   52946885.9975607591919232678676  
2018-12-25   0.0012774935024315606868  105095512.4409270069022683228722  
2019-06-01   2.4863555093566076039412  365247693.7532182599201748404057  
2019-02-23  49.2251458998634794044778   71649670.1004637510880899538066  
2019-04-30   0.0858095345267421270372   82437954.9936523631109510203076  
2019-02-11   0.9935910140668017184215    39003560.467850782045778994318  
2019-04-12   0.9730289597600692399944  150012676.5258322226549653628072